In [3]:
#Imports
import keras #code only works with this import!?
import os
import glob
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
#import timm
from sklearn.model_selection import train_test_split
import json
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.regularizers import l1
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
from segmentation_models_pytorch import Unet
from tqdm import tqdm
from PIL import Image
import copy
import random  # For random sampling

seed = 42
random.seed(seed)
np.random.seed(seed)

#0. Set user
User="Mark" #Set to Mark if you are Mark XD
print(User)

Mark


In [4]:
#Set Parameters Mark

#1. Set parameters for data loading
if User=="Mark":
    
    load_with_split = True #Load data with split or without split use the exel file
    excel_path = "Annotation_Zebrafish_full.xlsx" #change if Load with split == True
    #Df_Zebrafish_with_splits.xlsx
    
    images_folder = "C:/Users/ma405l/Documents/Heidelberg_Schweregrad/Full_data/Raw_data_full_train"
    masks_folder = "C:/Users/ma405l/Documents/Heidelberg_Schweregrad/Full_data/Masked_images"
    #2. Set parameters for data processing
    target_size=(256,256)	#Size of the images for training
    num_classes = 4

    #3. Train, Val, Test Split
    train_size = 0.6
    val_size = 0.2
    test_size = 0.2
    label_name = "Curved"

    #4. Balance datasets
    balance_train = True
    balance_val = False
    balance_test = False

    Model_type = "CNN"

    # Segmentation
    Model_seg = True
    num_epochs_seg = 1
    criterion_seg = torch.nn.BCEWithLogitsLoss()
    seg_directory = "Models/Segmentation"
    seg_train = False

    # HP Tuning
    hp_tuning = True
    hp_dense_layer = [512]
    hp_lr = [2e-4]

    #Use trained model for mask segmentation of images
    use_seg_model = True
    trained_seg_model = f"{seg_directory}/Segmentation/seg_model.pth"

    #5 Transformer. Augmentation parameters
    trans_rotation_range = 45
    trans_width_shift_range = 0.2
    trans_height_shift_range = 0.2
    trans_zoom_range = 0.1
    trans_horizontal_flip  = False
    trans_fill_mode="nearest"

    #6 Transformer. Training parameters
    trans_learning_rate = 0.00008 # Define learing rate
    trans_num_epochs = 1 # Define the number of epochs

    #7 Transformer. Save parameters
    trans_log_directory = f"Models/Transformer"
    trans_model_name = "trans_model.pth"
    trans_metrics_name = "trans_metrics.txt"

    #5 CNN. Augmentation parameters
    cnn_rotation_range = 45
    cnn_width_shift_range = 0.2
    cnn_height_shift_range = 0.2
    cnn_zoom_range = 0.1
    cnn_horizontal_flip  = False
    cnn_fill_mode="nearest"

    #6 CNN. Training parameters
    
    train_from_scratch = False #Train from scratch (vgg_16) or use pre-trained model (already trained)
    
    cnn_learning_rate = 0.001
    cnn_loss = 'crossentropy'
    cnn_num_epochs = 1
    cnn_num_epochs_pre = 1
    dense_layer = 512
    dropout = 0.3

    #7 CNN. Save parameters
    cnn_log_directory = "Models/CNN"
    cnn_model_name = "vgg_16_model.keras"
    cnn_metrics_name = "vgg_16_metrics.json"

In [5]:
#1) Match the images and masks to the exel data. 
# Create Df with the following columns: Image, Mask Path, Sample, Fish_Num, Edema, Curved, Masked Image
if load_with_split==True:
    def process_fish_data(excel_path, images_folder, masks_folder):
        # Read the Excel file
        df = pd.read_excel(excel_path, dtype={'Sample': str, 'Fish_Num': int, 'Edema': str, 'Curved': str})

        # Convert Fish_Num to two-digit format (01, 02, ...)
        df['Fish_Num'] = df['Fish_Num'].apply(lambda x: f"{x:02d}")

        # Store results
        results = []

        for _, row in df.iterrows():
            sample = row['Sample']
            fish_num = row['Fish_Num']
            edema = row['Edema']
            curved = row['Curved']

            # Find the image
            image_pattern = os.path.join(images_folder, f"*pr_{sample}-{fish_num}*.jpg")
            image_files = glob.glob(image_pattern)

            # Find the mask
            mask_pattern = os.path.join(masks_folder, f"*pr_{sample}-{fish_num}*_mask.jpg")
            mask_files = glob.glob(mask_pattern)

            # Ensure exactly one match
            if len(image_files) != 1 or len(mask_files) != 1:
                print(f"Skipping Sample {sample}, Fish {fish_num}: Image or mask missing/multiple found.")
                continue

            image_path = image_files[0]
            mask_path = mask_files[0]

            # Load image and mask
            image = cv2.imread(image_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            if image is None or mask is None:
                print(f"Skipping {image_path} or {mask_path}: Unable to read file.")
                continue

            # Apply the mask: Everything outside the mask becomes black
            masked_image = cv2.bitwise_and(image, image, mask=mask)

            # Store in results list
            results.append([image_path, mask_path, sample, fish_num, edema, curved, masked_image])

        # Convert to DataFrame
        columns = ['Images', 'Masks', 'Sample', 'Fish_Num', 'Edema', 'Curved', 'Masked Images']
        result_df = pd.DataFrame(results, columns=columns)

        return result_df

    df_result = process_fish_data(excel_path, images_folder, masks_folder)
    #Delete Rows with NAW
    df_result = df_result[df_result[label_name] != "NAW"]
    # Convert label to integers
    df_result["Curved"] = df_result["Curved"].astype(int) 

Skipping Sample 2, Fish 21: Image or mask missing/multiple found.
Skipping Sample 3, Fish 19: Image or mask missing/multiple found.
Skipping Sample 3, Fish 29: Image or mask missing/multiple found.
Skipping Sample 6, Fish 13: Image or mask missing/multiple found.
Skipping Sample 6, Fish 26: Image or mask missing/multiple found.
Skipping Sample 7, Fish 08: Image or mask missing/multiple found.
Skipping Sample 7, Fish 09: Image or mask missing/multiple found.
Skipping Sample 8, Fish 05: Image or mask missing/multiple found.
Skipping Sample 11, Fish 18: Image or mask missing/multiple found.
Skipping Sample 13, Fish 22: Image or mask missing/multiple found.
Skipping Sample 14, Fish 10: Image or mask missing/multiple found.
Skipping Sample 14, Fish 11: Image or mask missing/multiple found.
Skipping Sample 18, Fish 27: Image or mask missing/multiple found.
Skipping Sample 19, Fish 09: Image or mask missing/multiple found.
Skipping Sample 26, Fish 20: Image or mask missing/multiple found.
Ski

In [6]:
# Drop the 'Masked Images' column
df_cleaned = df_result.drop(columns=['Masked Images'])

# Stratified split by 'Curved'
train_c, temp_c = train_test_split(df_cleaned, test_size=0.4, stratify=df_cleaned['Curved'], random_state=42)
val_c, test_c = train_test_split(temp_c, test_size=0.5, stratify=temp_c['Curved'], random_state=42)

# Assign split labels for 'Curved'
df_cleaned['split_by_curve'] = -1
df_cleaned.loc[df_cleaned.index.isin(train_c.index), 'split_by_curve'] = 0
df_cleaned.loc[df_cleaned.index.isin(val_c.index), 'split_by_curve'] = 1
df_cleaned.loc[df_cleaned.index.isin(test_c.index), 'split_by_curve'] = 2

# Stratified split by 'Edema'
train_e, temp_e = train_test_split(df_cleaned, test_size=0.3, stratify=df_cleaned['Edema'], random_state=42)
val_e, test_e = train_test_split(temp_e, test_size=1/3, stratify=temp_e['Edema'], random_state=42)

# Assign split labels for 'Edema'
df_cleaned['split_by_edema'] = -1
df_cleaned.loc[df_cleaned.index.isin(train_e.index), 'split_by_edema'] = 0
df_cleaned.loc[df_cleaned.index.isin(val_e.index), 'split_by_edema'] = 1
df_cleaned.loc[df_cleaned.index.isin(test_e.index), 'split_by_edema'] = 2

# Save the new Excel file
#Diese zeile hier anpassen, wo soll die exel datai gepseichert werden? und welche datei name?
output_path = os.path.join(os.getcwd(), "Df_Zebrafish_with_splits.xlsx")
df_cleaned.to_excel(output_path, index=False)

print(f"New Excel file saved at: {output_path}")

New Excel file saved at: c:\Users\ma405l\OneDrive\Dokumente\PhD\Projects\Zebra_Fish\Df_Zebrafish_with_splits.xlsx
